In [1]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.naive_bayes import BernoulliNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score,precision_score, recall_score

In [2]:
## 设置字符集，防止中文乱码
mpl.rcParams['font.sans-serif']=[u'simHei']
mpl.rcParams['axes.unicode_minus']=False

In [3]:
# jupyter展示图片，非内嵌显示
# tk: 显示出来，inline：内嵌显示，默认为inline
%matplotlib tk

In [4]:
# 1. 文件数据读取
df = pd.read_csv('../data/result_process02', sep=',')
# 如果有某值为nan，进行删除操作
df.dropna(axis=0, how='any', inplace=True)
df.head(10)

,label,has_date,jieba_cut_content,content_sema
0,1.0,1,非 财务 纠淼 牟 莆 窆 芾 - （ 沙盘 模拟 ） - - ...,8.456151
1,0.0,1,讲 的 是 孔子 后人 的 故事 。 一个 老 领导 回到 家乡 ...,7.486084
2,1.0,1,尊敬 的 贵 公司 ( 财务 / 经理 ) 负责人 您好 ！ 我 ...,7.175171
3,1.0,1,贵 公司 负责人 ( 经理 / 财务 ） 您好 ： 深圳市 华龙 公...,7.565682
4,1.0,1,这是 一封 HTML 格式 信件 ！ - - - - - - - ...,2.063409
5,1.0,1,TO ： 贵 公司 经理 、 财务 您好 ！ 深圳市 春洋 贸易 有...,7.143747
6,0.0,1,那 他 为什么 不 愿意 起诉 ， 既然 这样 了 ！ 起诉 后 ...,4.807568
7,1.0,1,尊敬 的 负责人 （ 经理 ／ 财务 ） ： 您好 ！ 我 是 深...,6.593684
8,1.0,1,您好 以下 是 特别 为 阁下 发 的 香港 信息 ( 图片 ...,7.611074
9,0.0,1,我 觉得 ， 负债 不要紧 ， 最 重要 的 是 能 负得起 这个 ...,7.041340


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 64284 entries, 0 to 64619
Data columns (total 4 columns):
label                64284 non-null float64
has_date             64284 non-null int64
jieba_cut_content    64284 non-null object
content_sema         64284 non-null float64
dtypes: float64(2), int64(1), object(1)
memory usage: 2.5+ MB


In [6]:
# 2. 数据分割
x_train, x_test, y_train, y_test = train_test_split(df[['has_date','jieba_cut_content','content_sema']], df['label'], test_size=0.2, random_state=0)
print("训练数据集大小:%d" % x_train.shape[0])
print("测试数据集大小:%d" % x_test.shape[0])

训练数据集大小:51427
测试数据集大小:12857


In [7]:
x_train.head(5)

,has_date,jieba_cut_content,content_sema
18810,1,尊敬 的 公司 （ 工厂 ） 经理 负责人 你好 ： 我 公司 是 ...,6.849239
61632,1,声音 变换器 ： 适用 于 不同 型号 的 手机 ， 电话 ， 网络...,7.242064
5822,1,http : / / tengyingge . blogchina . co...,8.289843
5881,1,刚 开始 ， 是 会 有点 不 习惯 的 。 慢慢来 。 ps ：...,7.063309
26338,1,尊敬 的 公司 （ 工厂 ） 经理 负责人 你好 ： 我 公司 是 ...,6.700159


In [8]:
# 3. 开始模型训练
# 3.1 特征工程，将文本数据转换为数值型数据
transfromer = TfidfVectorizer(norm='l2', use_idf=True)
svd = TruncatedSVD(n_components=20)
jieba_cut_content = list(x_train['jieba_cut_content'].astype('str'))
transfromer_model = transfromer.fit(jieba_cut_content)
df1 = transfromer_model.transform(jieba_cut_content)
svd_model = svd.fit(df1)
df2 = svd_model.transform(df1)

data = pd.DataFrame(df2)
print(data.head(5))
print(data.info())

         0         1         2         3         4         5         6   \
0  0.340289  0.220012 -0.237481 -0.039929 -0.070163  0.002505 -0.160855   
1  0.014968  0.064739  0.027002 -0.004669  0.008162  0.007429  0.016378   
2  0.011862  0.072032  0.047006  0.001432 -0.002097  0.035910  0.009125   
3  0.019714  0.117439  0.132445  0.060406 -0.103514 -0.074868 -0.012198   
4  0.346257  0.215336 -0.234033 -0.043106 -0.071078 -0.000055 -0.162685   

         7         8         9         10        11        12        13  \
0 -0.009606  0.201680 -0.010785 -0.199526  0.039608 -0.065746  0.430054   
1  0.045969  0.029438  0.000047  0.007738  0.023426  0.041738 -0.011230   
2 -0.012129 -0.016859  0.000085  0.001073 -0.003870 -0.002272 -0.003785   
3 -0.005375 -0.027441  0.000119 -0.016805 -0.013143 -0.003589  0.016023   
4 -0.008875  0.198958 -0.010605 -0.195275  0.037906 -0.062235  0.417887   

         14        15        16        17        18        19  
0 -0.475845  0.202106  0.095477 -0

In [9]:
# 3.2 数据合并
data['has_date'] = list(x_train['has_date'])
data['content_sema'] = list(x_train['content_sema'])
print(data.head(5))
print(data.info())

          0         1         2         3         4         5         6  \
0  0.340289  0.220012 -0.237481 -0.039929 -0.070163  0.002505 -0.160855   
1  0.014968  0.064739  0.027002 -0.004669  0.008162  0.007429  0.016378   
2  0.011862  0.072032  0.047006  0.001432 -0.002097  0.035910  0.009125   
3  0.019714  0.117439  0.132445  0.060406 -0.103514 -0.074868 -0.012198   
4  0.346257  0.215336 -0.234033 -0.043106 -0.071078 -0.000055 -0.162685   

          7         8         9      ...             12        13        14  \
0 -0.009606  0.201680 -0.010785      ...      -0.065746  0.430054 -0.475845   
1  0.045969  0.029438  0.000047      ...       0.041738 -0.011230  0.013240   
2 -0.012129 -0.016859  0.000085      ...      -0.002272 -0.003785 -0.003157   
3 -0.005375 -0.027441  0.000119      ...      -0.003589  0.016023  0.002182   
4 -0.008875  0.198958 -0.010605      ...      -0.062235  0.417887 -0.466218   

         15        16        17        18        19  has_date  content_sem

In [10]:
%%time
nb = BernoulliNB(alpha=1.0,binarize=0.0005)
model = nb.fit(data, y_train)

Wall time: 58.2 ms


In [11]:
# 4.1 对测试数据进行特征转换
jieba_cut_content_test = list(x_test['jieba_cut_content'].astype('str'))
data_test = pd.DataFrame(svd_model.transform(transfromer_model.transform(jieba_cut_content_test)))
data_test['has_date'] = list(x_test['has_date'])
data_test['content_sema'] = list(x_test['content_sema'])
print(data_test.head(5))
print(data_test.info())

          0         1         2         3         4         5         6  \
0  0.537283  0.009048 -0.087600  0.028966  0.000837 -0.032717 -0.157714   
1  0.210902  0.131676 -0.090071 -0.086932 -0.033320  0.039651  0.006967   
2  0.045151  0.277282  0.233901  0.051111 -0.073998 -0.016321  0.010456   
3  0.032907  0.163066  0.113489  0.002482  0.053748  0.008246  0.004315   
4  0.035278  0.065204  0.026075 -0.023748  0.072922 -0.005953 -0.003272   

          7         8         9      ...             12        13        14  \
0  0.009387 -0.134471  0.009602      ...       0.079591  0.000773 -0.017332   
1 -0.008212  0.011706  0.000151      ...      -0.025829  0.082517 -0.056400   
2 -0.014928  0.100495 -0.002823      ...      -0.066454 -0.036709 -0.030789   
3 -0.020574  0.118087 -0.002192      ...      -0.061132 -0.056873 -0.012746   
4 -0.007937  0.007662  0.000563      ...      -0.002760 -0.007197 -0.004672   

         15        16        17        18        19  has_date  content_sem

In [12]:
# 4.2 对测试数据进行预测
y_predict = model.predict(data_test)

In [13]:
# 5. 效果评估
print("准确率为:%.5f" % precision_score(y_test, y_predict))
print("召回率为:%.5f" % recall_score(y_test, y_predict))
print("F1值为:%.5f" % f1_score(y_test, y_predict))

准确率为:0.94667
召回率为:0.98937
F1值为:0.96755
